In [41]:
import pandas as pd
from collections import Counter
import re

In [67]:
import warnings
warnings.simplefilter('ignore')

In [64]:
COMSBSR = pd.read_csv('../../output/UNITED_2020-04-23.csv').\
    assign(comm_direct = lambda df: df['comm_parent_id'].str.contains('t3_')).\
    set_index(['subreddit','subm_id','comm_id'])
COMSBSR['comm_body'] = COMSBSR['comm_body'].astype(str)
COMSBSR.shape

(1377155, 20)

In [65]:
Counter(COMSBSR.comm_direct)

Counter({True: 800619, False: 576536})

In [3]:
from IPython.core.display import display, HTML

def highlight_re(text, regex):
    htmld = re.sub(r'('+regex+')', '<mark>' + r'\1' + '</mark>', text.lower())
    display(HTML(htmld))
    
def print_submission(i, index, row):
    print('\n\n\nr/%s \n--------------------' % (index[0]))
    print('Published: %s' % row.subm_created[:20])
    print('%d | SUBM: %10s | COMM: %10s\n--------------------' % (i, index[1], index[2]))
    highlight_re('<b>'+row.subm_title+'</b>', covid_regex)
    print('URL: %s\nSubmission score: %d' % (row.subm_url, row.subm_score))

def print_comment(i, index, row, flag_no, limit = 1000):
    print('\nFLAG #%d (score = %d)' % (flag_no, row.comm_score))
    highlight_re(row.comm_body[:limit], fake_regex)    

In [39]:
covid_regex = '|'.join(pd.read_csv('../../input/keywords_covid.txt',header=None).loc[:,0].\
                  str.replace("\\(|\\)",r"").str.replace(r"\*",r""))#r"\\w*"))
fake_regex = '(fake |false).{0,15}(new\\w|info\\w|hood\\w)'+'|'+\
             '(dis|mis|mal)(info\\w|leading)'+'|'+\
             '\\bbs\\b|\\bbullshit\\b'+\
             '(not ).{0,5}(true\\b|verif\\w|real\\b)'+'|'+\
             '(unreli\\w+|unverifi\\w+|untrustw\\w+|fake\\w+)( source)'
fake_regex_no = '((no|not) .{0,10}('+fake_regex+'))'+'|(users often report submissions from this site)'

Extract comments with fake news regex

In [59]:
highlight_re('some americans are misinformed about the coronavirus. how responsible is the media?', fake_regex)

In [54]:
bool(re.search(fake_regex, 'misinformed'))

True

In [66]:
COMM_WITH_FLAGS = COMSBSR.\
    query('comm_body.str.contains("'+fake_regex+'")').\
    query('not comm_body.str.contains("'+fake_regex_no+'")').\
    query('not subm_title.str.contains("'+fake_regex+'")').\
    query('comm_direct')
COMM_WITH_FLAGS.shape

(6398, 20)

In [81]:
Counter(COMM_WITH_FLAGS.subm_title.str.contains('fake news'))

Counter({False: 6398})

In [68]:
COMM_WITH_FLAGS['subm_nflags'] = COMM_WITH_FLAGS.groupby(level = [0,1]).comm_body.transform('count')
#Counter(COMM_WITH_FLAGS['subm_nflags'])

In [73]:
COMM_WITH_FLAGS_latest1000 = COMM_WITH_FLAGS.sort_values('subm_created',ascending=False)[:1000]
COMM_WITH_FLAGS_min3 = COMM_WITH_FLAGS.query('subm_nflags>2').sort_values('subm_created',ascending=False)
COMM_WITH_FLAGS_min3.shape

(1765, 21)

#### Print selection

In [74]:
current_submission_id = 'none'
for i,(index,row) in enumerate(COMM_WITH_FLAGS_latest1000.iterrows()):
    if current_submission_id==index[1]:
        flag_no += 1
    else:
        print_submission(i,index,row)
        flag_no = 1
    current_submission_id=index[1]
    print_comment(i,index,row,flag_no)




r/Coronavirus 
--------------------
Published: 2020-04-21 21:48:51
0 | SUBM:  t3_g5m85z | COMM: t1_fo45vzs
--------------------


URL: https://www.marketwatch.com/story/hydroxychloroquine-treatment-for-covid-19-shows-no-benefit-and-more-deaths-in-va-study-2020-04-21?mod=home-page
Submission score: 7

FLAG #1 (score = -2)





r/Coronavirus 
--------------------
Published: 2020-04-21 21:47:03
1 | SUBM:  t3_g5m6wx | COMM: t1_fo5n5mk
--------------------


URL: https://thehill.com/policy/healthcare/493931-largest-analysis-of-hydroxychloroquine-use-finds-no-benefit-increased
Submission score: 91

FLAG #1 (score = 1)



FLAG #2 (score = 1)


...

URL: https://dallas.culturemap.com/news/city-life/04-17-20-governor-greg-abbott-reopen-texas-retail-state-parks/
Submission score: 63

FLAG #1 (score = 9)





r/technology 
--------------------
Published: 2020-04-18 02:32:48
847 | SUBM:  t3_g3e3y1 | COMM: t1_fo28io2
--------------------


URL: https://www.businessinsider.com/amazon-attempted-shut-down-warehouse-conditions-protest-deleted-calendar-invite-2020-4
Submission score: 200

FLAG #1 (score = 1)





r/Coronavirus 
--------------------
Published: 2020-04-18 02:25:42
848 | SUBM:  t3_g3e01v | COMM: t1_fnqpzol
--------------------


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [46]:
print('Percent of comments containing fake regex: %.2f' % (100*COMM_WITH_FLAGS.shape[0]/COMSBSR.shape[0])+'%')

Percent of comments containing fake regex: 0.74%


In [75]:
current_submission_id = 'none'
for i,(index,row) in enumerate(COMM_WITH_FLAGS_min3.iterrows()):
    if current_submission_id==index[1]:
        flag_no += 1
    else:
        print_submission(i,index,row)
        flag_no = 1
    current_submission_id=index[1]
    print_comment(i,index,row,flag_no)




r/Coronavirus 
--------------------
Published: 2020-04-21 21:47:03
0 | SUBM:  t3_g5m6wx | COMM: t1_fo5us6w
--------------------


URL: https://thehill.com/policy/healthcare/493931-largest-analysis-of-hydroxychloroquine-use-finds-no-benefit-increased
Submission score: 91

FLAG #1 (score = 1)



FLAG #2 (score = 2)



FLAG #3 (score = 1)



FLAG #4 (score = 0)


...




r/worldnews 
--------------------
Published: 2020-04-01 03:36:31
1761 | SUBM:  t3_fsq0qp | COMM: t1_fm40w16
--------------------


URL: https://www.businessinsider.com/coronavirus-italy-russia-military-convoy-supplies-useless-pr-stunt-2020-3
Submission score: 13

FLAG #1 (score = 1)



FLAG #2 (score = 1)



FLAG #3 (score = 1)



FLAG #4 (score = 1)


### Non-POS flags

In [103]:
import functions_pos

are flags

In [124]:
subjects = ['you','it','this','that','article','title','headline','header']
predicates = ['be','spread','stop','call',\
              'look','sound','feel','seem','smell']
fullobjects = ['disinformation','misinformation','misleading','bullshit','bs','falsehood','fabrication']
partialobjects = ['news','information','title','source','article','header']
partialattributes = ['fake','false','misleading','untrustworthy','bogus','bullshit']
###
fake_regex_wrds = 'fake|false\\w*|bogus|'+\
                  'mislead\\w*|disinf\\w*|misinf\\w*|malinf\\w*|'+\
                  'bullshit|\\bbs\\b|'+\
                  'news|info\\w*|title|article|source|submission'

In [125]:
are_flags = \
['how reliable is this source? ',
 'that’s the type of reactions this and many other misleading headlines do during times like these.',
 'this is how fake news is propagated.',
 'here comes the disinformation campaign',
 'the url sounds like one of those russian disinformation/dissension websites, doesn’t it?',
 'header is misleading.',
 'yeah cnn is a pretty unreliable source',
 'grade a example of fake news',
 'typical cnn fake news',
 "so i'm gonna guess this is a fake news article"]

In [126]:
extract_pos(' '.join(are_flags))

[('how', 'how', 'ADV', 'WRB', 'advmod', [reliable, is], []),
 ('reliable', 'reliable', 'ADJ', 'JJ', 'acomp', [is], [how]),
 ('is', 'be', 'AUX', 'VBZ', 'ROOT', [], [reliable, source, ?]),
 ('this', 'this', 'DET', 'DT', 'det', [source, is], []),
 ('source', 'source', 'NOUN', 'NN', 'nsubj', [is], [this]),
 ('?', '?', 'PUNCT', '.', 'punct', [is], [ ]),
 (' ', ' ', 'SPACE', '_SP', '', [?, is], []),
 ('that', 'that', 'DET', 'DT', 'ROOT', [], [’s, type, .]),
 ('’s', '’', 'VERB', 'VBZ', 'punct', [that], []),
 ('the', 'the', 'DET', 'DT', 'det', [type, that], []),
 ('type', 'type', 'NOUN', 'NN', 'appos', [that], [the, of, this, do]),
 ('of', 'of', 'ADP', 'IN', 'prep', [type, that], [reactions]),
 ('reactions', 'reaction', 'NOUN', 'NNS', 'pobj', [of, type, that], []),
 ('this', 'this', 'DET', 'DT', 'appos', [type, that], [and, headlines]),
 ('and', 'and', 'CCONJ', 'CC', 'cc', [this, type, that], []),
 ('many', 'many', 'ADJ', 'JJ', 'amod', [headlines, this, type, that], []),
 ('other', 'other', 'A

In [127]:
#Verb not root, but ccomp "guess/think this is fake news"

In [128]:
[fake_sent_pos(nlp(s)) for s in are_flags]

[False,
 False,
 False,
 False,
 False,
 'header is misleading',
 False,
 False,
 'fake news',
 False]

are not flags

In [129]:
are_not_flags = \
[
    'fake news! lol \*insert eye roll',
    'ppeople will still call this fake news. eople will still call this fake news.',
    'this is just fake news. /s',
    "experts? that's fake news. trump says it's going fine. ",  
]

In [130]:
[fake_sent_pos(nlp(s)) for s in are_not_flags]

['fake news', False, 'this is fake news', "that 's fake news"]